In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv('OPENAI_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# First experiment

In [ ]:
from utils.constants import STAGE_DIR

with open(STAGE_DIR / '01_sentences.txt', 'r') as f:
    sentences = f.readlines()

In [4]:
sentences = '''
Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva

Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables

Si el representante legal de una persona de capacidad limitada la autoriza expresa o tácitamente para realizar un trabajo, queda esta también autorizada para ejercitar los derechos y cumplir los deberes que se derivan de su contrato y para su cesación

Podrán contratar la prestación de su trabajo Los extranjeros, de acuerdo con lo dispuesto en la legislación específica sobre la materia

El contrato de trabajo se podrá celebrar por escrito o de palabra
'''

prompt = f'''
Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to develop a spanish event extraction task. The steps to achieve it are the following:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger, usually in the form of a verb. It can happen that a sentence relates more than one object entity with the same subject and event trigger. The sentences can contain entities and phrases that don't correspond to any classification. Also, the object can be separated from the subject and event trigger by these complements. The definitions of each category are the following:
    * event trigger: It refers to the action that is enforced by the legal text. It can be in a negative form, e.g. "no pueden".
    * subject entity: It refers to the entity that most follow, do the action of the event trigger.
    * object entity: It refers to the entity that is the receptor of the action. In the legal domain, it can be, for example, a right, a beneficiary from the action, an institution, a non-right, etc. 
    
    Example 1:
    Input: Cuando la relación laboral sea de duración superior a cuatro semanas, el empresario deberá informar por escrito, en los términos y plazos que se establezcan reglamentariamente, al trabajador sobre los elementos esenciales del contrato y las principales condiciones de ejecución de la prestación laboral, siempre que tales elementos y condiciones no figuren en el contrato de trabajo formalizado por escrito.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá informar por escrito"
        
    Example 2:
    Input: La denegación de la solicitud deberá ser notificada por el empresario al trabajador por escrito y de manera motivada.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá ser notificada"

3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
Note that it can be repeated sentences because of the different subject-event-object combinations.

The set of sentences in spanish to use is the following:

```
{sentences}
```
'''

In [6]:
response = get_completion(prompt)

'Output:\n```\n[\n    {\n        "sentence": "Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva",\n        "subject": "trabajadores",\n        "object": "Negociación colectiva",\n        "event": "tienen como derechos básicos"\n    },\n    {\n        "sentence": "Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables",\n        "subject": "trabajadores menores de dieciocho años",\n        "object": "trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación",\n        "event": "no podrán realizar"\n    },\n    {\n        "sentenc

In [7]:
print(response)

Output:
```
[
    {
        "sentence": "Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva",
        "subject": "trabajadores",
        "object": "Negociación colectiva",
        "event": "tienen como derechos básicos"
    },
    {
        "sentence": "Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables",
        "subject": "trabajadores menores de dieciocho años",
        "object": "trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación",
        "event": "no podrán realizar"
    },
    {
        "sentence": "Si el repres

# Experiments with `PlanTL-GOB-ES/RoBERTalex`

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, FillMaskPipeline
from pprint import pprint
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"¡Hola <mask>!"
res_hf = pipeline(text)
pprint([r['token_str'] for r in res_hf])

In [ ]:
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"El contrato de trabajo se podrá <mask>"
res_hf = pipeline(text)

pprint([r['token_str'] for r in res_hf])

In [ ]:
print(res_hf)

In [ ]:
seed = f"El contrato de trabajo se podrá <mask>"
i = 10
while i:
    tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model.eval()
    pipeline = FillMaskPipeline(model, tokenizer_hf)
    res_hf = pipeline(seed)
    max_score_sequence = max(res_hf, key=lambda x: x['score'])['sequence']
    seed = max_score_sequence + ' <mask>'
    i -= 1

print(seed)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
model = AutoModelForCausalLM.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
generator = pipeline('text-generation', tokenizer=tokenizer, model=model)
set_seed(42)
generator("Los  ", num_return_sequences=5)